In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [2]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')
# df = df.head(500000)
# df.sample()

In [3]:
x = 0.00
def stockDevelopement(row):
    return row['close']/row['open']
    

In [4]:
df['stockPrice_Change'] = df.apply(stockDevelopement, axis=1)

In [5]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPrice_Change"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPrice_Change':'_stockPrice_Change'})

In [6]:
df = df.set_index('id')
df = df.sort_index()

df.head()

,_headline,_stock,_headline_cleaned,_open,_close,_stockPrice_Change
id,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'frid...",92.13,90.38,0.981005
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'wedn...",90.65,90.49,0.998235
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.998840
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stocks.n.01', 'travel.v.01', 'friday.n.01', ...",85.00,84.98,0.999765
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,0.999765


In [7]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [8]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [9]:
df_set = df['_headline_sentence'].tolist()

In [10]:
df.shape

(161478, 7)

In [11]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', smooth_idf=True, use_idf=True)

count_wm = count_vectorizer.fit_transform(df_set)
tfidf_wm = tfidf_vectorizer.fit_transform(df_set)

count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=df.index, columns=tfidf_tokenz)

In [12]:
# # export the Matrixes because of some Allocation error sometimes
# df_countvect.to_csv('../data/df_countvect.csv', index=False)
# df_tfidfvect.to_csv('../data/df_tfidfvect.csv', index=False)

In [13]:
cols_to_delete = []
for column in df_tfidfvect:
    count = df_tfidfvect[df_tfidfvect[column] > 0].shape[0]
    if count<5:
        cols_to_delete.append(column)

print(len(cols_to_delete))
print(df_tfidfvect.shape[1])

4234
8927


In [14]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)

In [15]:
# mergred = pd.read_csv('../data/analyst_ratings_processed_final_tfidf_merged.csv')#
mergred =df.merge(df_tfidfvect, on='id')

In [16]:
pd.set_option('display.max_colwidth', None)

mergred.head(2)

,_headline,_stock,_headline_cleaned,_open,_close,_stockPrice_Change,_headline_sentence,aare,aaron,ability,...,york,young,young_person,yuan,yuma,zealand,zebra,zero,zion,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,0.981005,stocks hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,0.998235,stocks hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
import numpy as np

In [18]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [30, 37, 45, 53, 61, 68, 76, 84, 92, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [19]:
from sklearn.ensemble import RandomForestRegressor


In [20]:
# rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2)

In [21]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,_stockPrice_Change,_headline_sentence,aare,aaron,ability,...,york,young,young_person,yuan,yuma,zealand,zebra,zero,zion,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,0.981005,stocks hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,0.998235,stocks hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.998840,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stocks.n.01', 'travel.v.01', 'friday.n.01', 'session.n.01']",85.00,84.98,0.999765,stocks travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88",A,"['security.n.01', 'neutral.n.01', 'technology.n.01', 'raise.n.01', 'monetary_value.n.01', 'target.n.01']",85.00,84.98,0.999765,security neutral technology raise monetary_value target,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,_stockPrice_Change,_headline_sentence,aare,aaron,ability,...,york,young,young_person,yuan,yuma,zealand,zebra,zero,zion,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,0.981005,stocks hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stocks.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,0.998235,stocks hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.998840,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stocks.n.01', 'travel.v.01', 'friday.n.01', 'session.n.01']",85.00,84.98,0.999765,stocks travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88",A,"['security.n.01', 'neutral.n.01', 'technology.n.01', 'raise.n.01', 'monetary_value.n.01', 'target.n.01']",85.00,84.98,0.999765,security neutral technology raise monetary_value target,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Fit the random search model
# rf_random.fit(X, y)


In [24]:
# rf_random.best_params_

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
splitted_set = train_test_split(mergred, test_size=0.3)
train_set = splitted_set[0]
test_set = splitted_set[1]

In [27]:
X = train_set.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', '_stockPrice_Change', '_headline_sentence'])
Y = train_set['_stockPrice_Change']

In [28]:
feature_words = list(X)
dflist = pd.DataFrame(feature_words)
print(dflist)
dflist.to_csv('../data/tfidf_features.csv', index=False)

                       0
0                   aare
1                  aaron
2                ability
3                   able
4     abominable_snowman
...                  ...
4688             zealand
4689               zebra
4690                zero
4691                zion
4692                zone

[4693 rows x 1 columns]


In [29]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor( n_estimators=37, min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=10, bootstrap=True)
regressor.fit(X, Y)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                      n_estimators=37)

In [31]:
test_set = test_set.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', '_headline_sentence'])

In [34]:
test_set["_prediction"] = test_set.apply(lambda row: regressor.predict([row[1:]]), axis=1)

In [35]:
def standardization(quotient):
    if quotient >= 1:
        return int(1)
    if quotient < 1:
        return int(0)

In [36]:
test_set['_stockPrice_Change'] = test_set._stockPrice_Change.apply(standardization)
test_set['_prediction'] = test_set._prediction.apply(standardization)

In [ ]:
# y_pred_bin = []
# for value in y_pred:
#     if value >= 1:
#         y_pred_bin.append(1)
#     elif value < 1:
#         y_pred_bin.append(0)

In [42]:
from sklearn import metrics
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(test_set['_stockPrice_Change'], test_set['_prediction']))
print('Mean Squared Error:', metrics.mean_squared_error(test_set['_stockPrice_Change'], test_set['_prediction']))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_set['_stockPrice_Change'], test_set['_prediction'])))

Mean Absolute Error: 0.43625629592932047
Mean Squared Error: 0.43625629592932047
Root Mean Squared Error: 0.6604970067527335


In [43]:
print('Accuracy Score:', metrics.accuracy_score(test_set['_stockPrice_Change'], test_set['_prediction']))

Accuracy Score: 0.5637437040706795


In [ ]:
pd.read_csv('../data/tfidf_features.csv')['0']

0                     aare
1                    aaron
2                  ability
3                     able
4       abominable_snowman
               ...        
4688               zealand
4689                 zebra
4690                  zero
4691                  zion
4692                  zone
Name: 0, Length: 4693, dtype: object

In [ ]:
def predictHeadline(hl):

    count_hl = len(list(hl.split()))
    single_headline_tfidf = pd.DataFrame()
    feature_list = list(X)
    for feature in feature_list:
        idf_value = tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[feature]]
        count_feature = hl.count(feature)
        single_headline_tfidf[feature] = [(count_feature / count_hl) * idf_value]
    print(regressor.predict(single_headline_tfidf))
    
predictHeadline("stock very high best apple low")


C:\Users\DOMINI~1\AppData\Local\Temp/ipykernel_14876/1561814447.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  single_headline_tfidf[feature] = [(count_feature / count_hl) * idf_value]


[0.03892093]


In [ ]:
import joblib
joblib.dump(regressor, './deployment/tfidf-model.pkl')

['./deployment/tfidf-model.pkl']